Kita h5pyd is still on 0.3.2, so need to pip install latest version of h5pyd:
```
pip install git+https://github.com/HDFGroup/h5pyd.git
```

In [1]:
# !pip install git+https://github.com/HDFGroup/h5pyd.git 

In [2]:
%matplotlib inline
import h5pyd as h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
print(h5py.__version__)

0.4.1


In [4]:
! hsls -H -v /home/jreadey/ESIP/

jreadey@hdfgroup.org                 folder   2019-05-04 14:15:37 /home/jreadey/ESIP/
jreadey@hdfgroup.org   104.1K        domain   2019-06-06 20:12:39 /home/jreadey/ESIP/adcirc_01.nc
2 items


## Try reading with h5pyd (0.4.0 version)

In [5]:
url_hsds = "http://hsdshdflab.hdfgroup.org/home/jreadey/ESIP/adcirc_native.nc"
f = h5py.File("/home/jreadey/ESIP/adcirc_01.nc", "r")

In [6]:
zeta = f['zeta']

In [7]:
zeta.chunks

(10, 141973)

In [8]:
zeta.shape

(720, 9228245)

Let's try reading one chunk of data

In [9]:
n = 141973

In [10]:
%time arr = f['zeta'][:320, 2*n:3*n]

CPU times: user 792 ms, sys: 1.25 s, total: 2.04 s
Wall time: 12 s


How many chunks are there?

In [11]:
f['zeta']

<HDF5 dataset "zeta": shape (720, 9228245), type "<f8">

In [12]:
nchunks = np.prod(zeta.shape) / np.prod(zeta.chunks)
print(nchunks)

4680.0


In [13]:
f['zeta']

<HDF5 dataset "zeta": shape (720, 9228245), type "<f8">

In [14]:
np.prod(zeta.chunks)*8/1e6

11.35784

In [15]:
11*32/7

50.285714285714285

Is this the theoretical time to read all the chunks using 16 cores? 

In [16]:
total_seconds = 0.327 * nchunks / 16
print(total_seconds)

95.64750000000001


## Now try reading with xarray (using h5netcdf + h5pyd)

In [21]:
import xarray as xr

In [22]:
url_hsds = 'http://hsdshdflab.hdfgroup.org/home/jreadey/ESIP/adcirc_01.nc'
url_hsds = "http://hsdshdflab.hdfgroup.org/home/jreadey/ESIP/adcirc_native.nc"

In [23]:
%%time
ds_hsds = xr.open_dataset(url_hsds, engine='h5netcdf', chunks={'time':10, 'node':141973})

OSError: [Errno 403] Forbidden

In [20]:
ds_hsds['zeta']

NameError: name 'ds_hsds' is not defined

In [ ]:
ind = 7454940

In [ ]:
%%time
ds_hsds['zeta'][:,ind].plot()

## xarray with local Dask cluster (since my docker container doesn't have h5pyd)

In [ ]:
from dask.distributed import Client, progress, LocalCluster, progress

In [ ]:
cluster = LocalCluster()

In [ ]:
client = Client(cluster)

In [ ]:
client

In [ ]:
ds_hsds1 = xr.open_dataset(url_hsds, engine='h5netcdf', chunks={'time':10, 'node':141973})

In [ ]:
ds_hsds1['zeta']

In [ ]:
%%time
ds_hsds1['zeta'][:,ind].plot()

In [ ]:
%%time
max_var = ds_hsds1['zeta'].max(dim='time').compute()

In [ ]:
%time xr_dset = ds_hsds1['zeta'][:,ind]